In [1]:
import os
import subprocess
import MDAnalysis as mda
import numpy as np

/home/ys/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
/home/ys/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


In [2]:
# Extract the suitable strfiles
# Data
data_path = '/home/ys/SongYang2/GoMartini3/Beta2AR/Martini/Data/repeat-7'
strfile = os.path.join(data_path, "md_start.pdb")
trjfile_processed = os.path.join(data_path, 'md_rottrans.xtc')
trjfile_raw = os.path.join(data_path, "md_whole.xtc")

# Extracting Process
u_processed = mda.Universe(strfile, trjfile_processed)
u_raw = mda.Universe(strfile, trjfile_raw)
print(u_processed.trajectory.n_frames)
print(u_raw.trajectory.n_frames)

# Calculae the distance array
def Distance_analysis(u):
    sel1 = u.select_atoms('name BB and resid 131')
    sel2 = u.select_atoms('name BB and resid 272')
    data = []
    for ts in u.trajectory:
        dist = np.linalg.norm(sel1.positions-sel2.positions, axis=1)[0]
        data.append([ts.frame, ts.time, dist])

    return np.array(data)

dist_array = Distance_analysis(u_processed)
dist_min = dist_array[:,2].min()
dist_max = dist_array[:,2].max()

# Get the frame according to the distance
frames_sel = []
for dist in np.linspace(dist_min, dist_max, 10):
    frame_sel = np.abs(dist_array[:,2] - dist).argmin()
    frames_sel.append(frame_sel)
    print(frame_sel, dist_array[frame_sel, 2], dist)

# Write the gro file
for i, frame in enumerate(frames_sel):
    u_raw.trajectory[frame]
    u_raw.atoms.write(f'Beta2AR_No{i}.gro')

/home/ys/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/ys/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/ys/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


40001
40001
33846 8.103084564208984 8.103084564208984
29241 9.172789573669434 9.173453225029839
21709 10.244023323059082 10.243821885850695
26981 11.344161987304688 11.31419054667155
3088 12.382648468017578 12.384559207492405
18617 13.45547866821289 13.45492786831326
7650 14.525154113769531 14.525296529134115
13548 15.595829963684082 15.595665189954971
6592 16.67243194580078 16.666033850775825
6551 17.73640251159668 17.73640251159668


In [ ]:
def NPTProcess(strfile, top='system_Inactive.top'):
    result = subprocess.run(f"""
    gmx grompp -f em.mdp -c {strfile} -p {top} -o em.tpr -n index.ndx
    gmx mdrun -deffnm em -v  -nt 8

    gmx grompp -f npt.mdp -c em.gro -p {top} -o npt.tpr -r em.gro -n index.ndx
    gmx mdrun -deffnm npt -v -nt 8

    echo a_209 0|gmx trjconv -s npt.tpr -f npt.gro -o {strfile} -pbc atom -center -n index.ndx
    echo 1 0|gmx trjconv -s npt.tpr -f {strfile} -o {strfile} -pbc mol -center -n index.ndx
    rm *.edr em.gro  npt.gro *.log *.tpr *.trr *.cpt  *.xtc *.pdb npt.gro em.gro mdout.mdp
    rm *#
                """, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"{strfile} finished!")
        pass
    else:
        print(f"Error: {strfile} Return code: {result.returncode}")
        print("Standard Output:")
        print(result.stdout.decode())
        print("Standard Error:")
        print(result.stderr.decode())

In [5]:
# Set velocity (NPT process)
for i, frame in enumerate(frames_sel):
    NPTProcess(f'Beta2AR_No{i}.gro', top='system_Inactive.top')




Beta2AR_No0.gro finished!
Beta2AR_No1.gro finished!
Beta2AR_No2.gro finished!
Beta2AR_No3.gro finished!
Beta2AR_No4.gro finished!
Beta2AR_No5.gro finished!
Beta2AR_No6.gro finished!
Beta2AR_No7.gro finished!
Beta2AR_No8.gro finished!
Beta2AR_No9.gro finished!


In [6]:
aaa

NameError: name 'aaa' is not defined

In [7]:
def NPTProcess2(strfile, top='system_Inactive.top'):
    result = subprocess.run(f"""
    gmx grompp -f em.mdp -c {strfile} -p {top} -o em.tpr -n index.ndx
    echo a_209 0|gmx trjconv -s em.tpr -f {strfile} -o {strfile} -pbc atom -center -n index.ndx
    echo 1 0|gmx trjconv -s em.tpr -f {strfile} -o {strfile} -pbc mol -center -n index.ndx
    rm *.edr em.gro  npt.gro *.log *.tpr *.trr *.cpt  *.xtc *.pdb npt.gro em.gro mdout.mdp
    rm *#
                """, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"{strfile} finished!")
        pass
    else:
        print(f"Error: {strfile} Return code: {result.returncode}")
        print("Standard Output:")
        print(result.stdout.decode())
        print("Standard Error:")
        print(result.stderr.decode())

# Set velocity (NPT process)
for i, frame in enumerate(frames_sel):
    NPTProcess2(f'Beta2AR_No{i}.gro', top='system_Inactive.top')




Beta2AR_No0.gro finished!
Beta2AR_No1.gro finished!
Beta2AR_No2.gro finished!
Beta2AR_No3.gro finished!
Beta2AR_No4.gro finished!
Beta2AR_No5.gro finished!
Beta2AR_No6.gro finished!
Beta2AR_No7.gro finished!
Beta2AR_No8.gro finished!
Beta2AR_No9.gro finished!
